# 🏛️ Asistente Legal Colombiano con LangChain + LLM

**Proyecto:** Sistema de consulta inteligente de leyes y documentos normativos colombianos

**Autores:** [Agregar nombres del equipo aquí]

**Fecha:** Octubre 2025

---

## 📋 Objetivo del Proyecto

Desarrollar un asistente conversacional que permita consultar la Constitución Política de Colombia y otros documentos legales mediante:
- **Búsqueda semántica** con RAG (Retrieval-Augmented Generation)
- **Procesamiento de lenguaje natural** con LLMs
- **Agentes inteligentes** que razonan y seleccionan herramientas
- **Interfaz interactiva** con Jupyter widgets

---

## 🎯 Componentes Implementados

✅ **a) Mensajes y plantillas** - PromptTemplates con variables dinámicas  
✅ **b) Parsers estructurados** - Validación de salidas con JSON schema  
✅ **c) Loaders** - Carga desde archivos locales, PDFs y URLs  
✅ **d) Vector DB + RAG** - Embeddings y búsqueda semántica  
✅ **e) Memoria conversacional** - Contexto multi-turno con LangChain  
✅ **f) Agentes ReAct** - Razonamiento y selección de herramientas  
✅ **g) Transformación de documentos** - Splitters y preprocesamiento  

---

## 🚀 Demo Rápida

**Flujo de uso:**
1. Cargar documentos legales (PDF/TXT)
2. Indexar con embeddings en vector database
3. Hacer preguntas en lenguaje natural
4. Obtener respuestas con referencias a artículos específicos

**Instrucciones:** Ejecuta las celdas en orden secuencial (⬇️)

In [30]:
##!uv add "langchain[openai]" dotenv ipywidgets chromadb faiss-cpu PyPDF2

## installar dependencias con pip
#%pip install -q "langchain[openai]" python-dotenv ipywidgets chromadb faiss-cpu PyPDF2
#!pip install groq langchain-groq


In [2]:
# Imports y configuración inicial
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Import the LLM provider module
from llm_providers import get_llm_client, LLMProviderFactory

# Get LLM provider from environment (default: groq)
provider = os.getenv('LLM_PROVIDER', 'groq').lower()
print(f"Available providers: {LLMProviderFactory.get_available_providers()}")
print(f"Using provider: {provider}")

# Initialize LLM client based on provider
llm = get_llm_client(provider)
print(f"LLM client initialized successfully!")

# Prompt template de ejemplo
prompt = PromptTemplate(
    input_variables=["question"],
    template="Eres un asistente legal colombiano. Responde: {question}"
)


Available providers: ['openai', 'openrouter', 'ollama', 'groq']
Using provider: openrouter
LLM client initialized successfully!


In [3]:
# Loaders: widget FileUpload y helpers para parsear archivos subidos y leer `data/`
import ipywidgets as widgets
from IPython.display import display
import os
from io import BytesIO

# Widget para subir archivos
uploader = widgets.FileUpload(accept='.txt,.md,.pdf', multiple=True)
display(widgets.VBox([widgets.Label("Sube archivos legales (.pdf, .txt, .md)"), uploader]))

# Output para previsualización
output = widgets.Output()
display(output)


def parse_uploaded_files(uploader_widget):
    """Devuelve una lista de dicts {'filename', 'text'} extraídos de los archivos subidos."""
    docs = []
    # La estructura de uploader.value depende del frontend; manejar ambos casos
    items = getattr(uploader_widget, 'value', {}) or {}
    # En algunos entornos items es lista, en otros dict
    try:
        iterator = items.items()
    except Exception:
        # intentamos tratar como lista
        iterator = [(f.get('name', f"file_{i}"), f) for i, f in enumerate(items)]

    for name, fileinfo in iterator:
        # fileinfo puede ser dict con 'content' o directamente bytes
        content = fileinfo.get('content') if isinstance(fileinfo, dict) else fileinfo
        text = ''
        if name.lower().endswith('.pdf'):
            try:
                from PyPDF2 import PdfReader
                reader = PdfReader(BytesIO(content))
                pages = [p.extract_text() or '' for p in reader.pages]
                text = '\n'.join(pages)
            except Exception as e:
                text = f"[ERROR] No se pudo leer PDF ({name}): {e}"
        else:
            try:
                if isinstance(content, bytes):
                    text = content.decode('utf-8')
                else:
                    text = str(content)
            except Exception as e:
                text = f"[ERROR] decodificando {name}: {e}"
        docs.append({'filename': name, 'text': text})
    return docs


def on_preview_clicked(b):
    with output:
        output.clear_output()
        docs = parse_uploaded_files(uploader)
        if not docs:
            print('No se han encontrado archivos subidos.')
            return
        for d in docs:
            print(f"--- {d['filename']} ---")
            print(d['text'][:1500])
            print('\n')

preview_btn = widgets.Button(description='Previsualizar archivos')
preview_btn.on_click(on_preview_clicked)
display(preview_btn)


def load_local_docs(data_dir='data'):
    """Lee archivos de `data/` y devuelve lista de dicts {'filename','text'}"""
    docs = []
    if not os.path.exists(data_dir):
        print(f"No existe la carpeta '{data_dir}'. Crea la carpeta y añade documentos de ejemplo.")
        return docs
    for fn in sorted(os.listdir(data_dir)):
        path = os.path.join(data_dir, fn)
        if not os.path.isfile(path):
            continue
        text = ''
        if fn.lower().endswith('.pdf'):
            try:
                from PyPDF2 import PdfReader
                reader = PdfReader(path)
                pages = [p.extract_text() or '' for p in reader.pages]
                text = '\n'.join(pages)
            except Exception as e:
                text = f"[ERROR] leyendo PDF {fn}: {e}"
        else:
            try:
                with open(path, 'r', encoding='utf-8') as f:
                    text = f.read()
            except Exception as e:
                text = f"[ERROR] leyendo {fn}: {e}"
        docs.append({'filename': fn, 'text': text})
    return docs

# Ejemplo de uso: docs = load_local_docs('data')
# print(docs[0]['text'][:500]) if docs else print('No hay documentos en data/')


Output()

Button(description='Previsualizar archivos', style=ButtonStyle())

## Sección: Loaders (carga de documentos)

En esta sección implementaremos los loaders que permiten: (1) subir archivos locales mediante un widget, y (2) cargar archivos desde la carpeta `data/` del repositorio. Más adelante convertiremos estos textos en chunks y los indexaremos en una vector DB.

Instrucciones rápidas:
- Usa el widget para subir `.pdf`, `.txt` o `.md`.
- Pulsa "Previsualizar archivos" para ver el texto extraído (o errores si falta una dependencia de PDF).

In [18]:
# Preprocesamiento: splitters y creación de chunks
# Carga documentos desde `data/`, aplica un splitter y guarda los chunks en `DOCUMENT_CHUNKS`.
from collections import Counter

# Intentar importar el splitter de LangChain; si la API cambia, manejar el fallback
try:
    from langchain.text_splitter import RecursiveCharacterTextSplitter
except Exception:
    try:
        # versión alternativa
        from langchain.text_splitter import CharacterTextSplitter as RecursiveCharacterTextSplitter
    except Exception:
        RecursiveCharacterTextSplitter = None


def create_chunks(docs, chunk_size=1000, chunk_overlap=200):
    """Devuelve lista de dicts {'source','chunk_index','text'}"""
    if RecursiveCharacterTextSplitter is None:
        raise ImportError("No se encontró RecursiveCharacterTextSplitter. Instala langchain actualizado.")
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    all_chunks = []
    for d in docs:
        text = d.get('text') or ''
        if not text.strip():
            # texto vacío o error al leer
            continue
        texts = splitter.split_text(text)
        for i, t in enumerate(texts):
            all_chunks.append({'source': d.get('filename', 'unknown'), 'chunk_index': i, 'text': t})
    return all_chunks

# Cargar documentos locales desde data/
DOCUMENTS = load_local_docs('data')
print(f"Documentos cargados: {len(DOCUMENTS)}")

# Crear chunks
DOCUMENT_CHUNKS = create_chunks(DOCUMENTS, chunk_size=1000, chunk_overlap=200) if DOCUMENTS else []
print(f"Total chunks generados: {len(DOCUMENT_CHUNKS)}")

# Mostrar conteo por documento y un ejemplo
if DOCUMENT_CHUNKS:
    per_doc = Counter([c['source'] for c in DOCUMENT_CHUNKS])
    for doc, cnt in per_doc.items():
        print(f" - {doc}: {cnt} chunks")
    print('\nEjemplo (primer chunk, 500 chars):')
    print(DOCUMENT_CHUNKS[0]['text'][:500])
else:
    print('No hay chunks (documentos vacíos o problema al leer los archivos).')

# Ahora `DOCUMENTS` y `DOCUMENT_CHUNKS` están listos para embeddings / indexado.


Documentos cargados: 1
Total chunks generados: 165
 - reglamento_academi_pregrado.pdf: 165 chunks

Ejemplo (primer chunk, 500 chars):
ACUERDO  No.186  
02 de  diciembre de 2005  
 
 
Por el cual  compila y actualiza   el Reglamento Académico Estudiantil de Pregrado.  
 
 
EL CONSEJO  SUPERIOR  DE LA UNIVERSIDAD  DE PAMPLONA,  EN USO DE  SUS 
ATRIBUCIONES  LEGALES,  en especial de las que le confiere el art ículo, y,  
Artículo 23 Literales a) y d)  
 
CONSIDERANDO:  
 
 
1. Que es función  del Consejo  Superior Universitario,  expedir  ó modificar  los 
Estatutos  y Regla mentos  de la Institución,  previo  concepto  de las in


## Sección: Vector Database (Opcional)

Esta sección es **opcional**. Si deseas mejorar la búsqueda semántica, puedes crear un vector database (Chroma o FAISS) con embeddings.

**Ventajas de usar vectordb:**
- Búsqueda semántica más precisa (entiende el significado, no solo palabras clave)
- Mejor recuperación de documentos relevantes
- Resultados más coherentes

**Si no usas vectordb:**
- El sistema funcionará con búsqueda por palabras clave (fallback automático)
- Es más rápido pero menos preciso semánticamente

Ejecuta la siguiente celda si quieres configurar embeddings con OpenAI o alternativas gratuitas.

In [19]:
# ===== SECCIÓN: Crear Vector Database con Embeddings =====
# Esta celda crea una base de datos vectorial para búsqueda semántica avanzada

from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document
import os

# Verificar que tenemos chunks disponibles
if not DOCUMENT_CHUNKS:
    print("⚠️ No hay chunks disponibles. Ejecuta primero la celda de preprocesamiento.")
    vectordb = None
else:
    print(f"📦 Preparando {len(DOCUMENT_CHUNKS)} chunks para indexación...")
    
    # Convertir chunks a formato Document de LangChain
    documents = []
    for chunk in DOCUMENT_CHUNKS:
        doc = Document(
            page_content=chunk['text'],
            metadata={
                'source': chunk['source'],
                'chunk_index': chunk['chunk_index']
            }
        )
        documents.append(doc)
    
    # Intentar crear embeddings y vectordb
    try:
        print("🔄 Creando embeddings (esto puede tardar 1-2 minutos)...")
        
        # Configurar embeddings según el proveedor
        provider = os.getenv('LLM_PROVIDER', 'groq').lower()
        
        if provider in ['openai', 'groq', 'openrouter']:
            # Usar embeddings de OpenAI (compatible con OpenRouter y Groq también)
            embeddings = OpenAIEmbeddings(
                openai_api_key=os.getenv('OPENAI_API_KEY', os.getenv('OPENROUTER_API_KEY'))
            )
        else:
            # Fallback a OpenAI estándar
            embeddings = OpenAIEmbeddings()
        
        # Crear Chroma vectorstore con persistencia
        vectordb = Chroma.from_documents(
            documents=documents,
            embedding=embeddings,
            persist_directory="./chroma_db"
        )
        
        print(f"✅ Vector database creado exitosamente!")
        print(f"   - Total documentos indexados: {len(documents)}")
        print(f"   - Directorio de persistencia: ./chroma_db")
        print(f"   - Embedding model: {embeddings.model}")
        
    except Exception as e:
        print(f"❌ Error al crear embeddings: {e}")
        print("   Se usará búsqueda por keywords como fallback")
        vectordb = None

# Verificar estado final
if vectordb is not None:
    print("\n🎉 Modo: BÚSQUEDA SEMÁNTICA (embeddings activos)")
else:
    print("\n⚠️ Modo: BÚSQUEDA POR KEYWORDS (fallback)")

📦 Preparando 165 chunks para indexación...
🔄 Creando embeddings (esto puede tardar 1-2 minutos)...
✅ Vector database creado exitosamente!
   - Total documentos indexados: 165
   - Directorio de persistencia: ./chroma_db
   - Embedding model: text-embedding-ada-002

🎉 Modo: BÚSQUEDA SEMÁNTICA (embeddings activos)
✅ Vector database creado exitosamente!
   - Total documentos indexados: 165
   - Directorio de persistencia: ./chroma_db
   - Embedding model: text-embedding-ada-002

🎉 Modo: BÚSQUEDA SEMÁNTICA (embeddings activos)


In [5]:
#%pip install -q --upgrade langchain-openai
#!uv add langchain-openai --upgrade
!uv add langchain-community

Resolved 157 packages in 845ms                                       
Resolved 157 packages in 845ms                                       
Installed 14 packages in 33ms.1                                  ░░░░░░░░░░░░░░░░░░░░ [0/0] Installing wheels...                                 ░░░░░░░░░░░░░░░░░░░░ [0/0] Installing wheels...                                 
 + aiohappyeyeballs==2.6.1
 + aiohttp==3.12.15
 + aiosignal==1.4.0
 + dataclasses-json==0.6.7
 + frozenlist==1.7.0
 + httpx-sse==0.4.1
 + langchain-community==0.3.30
 + marshmallow==3.26.1
 + multidict==6.6.4
 + mypy-extensions==1.1.0
 + propcache==0.3.2
 + pydantic-settings==2.11.0
 + typing-inspect==0.9.0
 + yarl==1.20.1
Installed 14 packages in 33ms.1                             
 + aiohappyeyeballs==2.6.1
 + aiohttp==3.12.15
 + aiosignal==1.4.0
 + dataclasses-json==0.6.7
 + frozenlist==1.7.0
 + httpx-sse==0.4.1
 + langchain-community==0.3.30
 + marshmallow==3.26.1
 + multidict==6.6.4
 + mypy-extensions==1.1.0
 + propcache=

In [20]:
##SECCION 5 PARSES 
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate

# Definir los campos esperados en la salida
response_schemas = [
    ResponseSchema(name="answer", description="Respuesta concisa, clara a la consulta legal o normativa en español."),
    ResponseSchema(name="citations", description="Lista de citas con formato [doc:filename, chunk:index]."),
    ResponseSchema(name="follow_up", description="true/false indicando si el usuario debería hacer una pregunta de seguimiento."),
]

# Crear el parser estructurado
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# Instrucciones para el modelo
format_instructions = output_parser.get_format_instructions()

# Prompt template que exige formato
parser_prompt = PromptTemplate(
    template="""
Eres un asistente legal-normativo colombiano. Responde siempre en formato JSON.

Pregunta: {question}

{format_instructions}
""",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
)

# Ejemplo de uso
question = "¿Cuáles son los requisitos para graduarse en la universidad de pamplona?"

prompt_text = parser_prompt.format(question=question)
response = llm.invoke(prompt_text)

print("=== Respuesta cruda del LLM ===")
print(response.content)

try:
    parsed = output_parser.parse(response.content)
    print("\n=== Respuesta parseada (dict válido) ===")
    print(parsed)
except Exception as e:
    print("\n[ERROR] No se pudo parsear la salida:", e)


=== Respuesta cruda del LLM ===
```json
{
	"answer": "Los requisitos generales para optar al título en la Universidad de Pamplona, de acuerdo con su Reglamento Estudiantil (Acuerdo No. 040 de 2016), son:\n\n1. Haber cursado y aprobado la totalidad de las asignaturas y actividades académicas contempladas en el plan de estudios del programa.\n2. Cumplir con el requisito de suficiencia en segunda lengua, de acuerdo con la reglamentación específica expedida por el Consejo Académico.\n3. Presentar y aprobar la opción de grado, según lo establecido en el plan de estudios y la reglamentación vigente (que puede ser, entre otras, trabajo de grado, pasantía, monografía, o curso de profundización).\n4. Haber cumplido con el requisito de la Prueba Saber Pro (antes ECAES).\n5. Estar a paz y salvo por todo concepto con la Universidad (financiera, bibliotecaria y documentalmente).\n6. Cancelar los derechos de grado correspondientes.\n7. Cumplir con los demás requisitos específicos establecidos por el

In [21]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Definir un prompt que incluya historial de conversación
memory_prompt = PromptTemplate(
    input_variables=["history", "question"],
    template="""
Eres un asistente legal-normativo colombiano.
Responde en español de forma clara y breve.

Historial de conversación:
{history}

Nueva pregunta:
{question}

Respuesta:
"""
)

# Inicializar la memoria
memory = ConversationBufferMemory(memory_key="history")

# Construir el chain con memoria
conversation_chain = LLMChain(
    llm=llm,
    prompt=memory_prompt,
    memory=memory,
)

# Ejemplo de interacción multi-turno
print("Turno 1:")
resp1 = conversation_chain.run("Soy un estudiante de Ingenieria de sistemas y me llamo Wilson")
print(resp1)

print("\nTurno 2 (con memoria):")
resp2 = conversation_chain.run("¿Como me llamo y que estudio?")
print(resp2)



Turno 1:
Hola Wilson. ¿En qué puedo ayudarte?

Turno 2 (con memoria):
Hola Wilson. ¿En qué puedo ayudarte?

Turno 2 (con memoria):
Te llamas **Wilson** y estudias **Ingeniería de Sistemas**.
Te llamas **Wilson** y estudias **Ingeniería de Sistemas**.


In [22]:
from langchain.agents import initialize_agent, Tool, AgentType
from langchain_core.prompts import PromptTemplate

# Definición básica de rag_answer para evitar NameError
import textwrap

def rag_answer(query: str, k: int = 3):
    """Función básica de RAG usando DOCUMENT_CHUNKS."""
    if 'DOCUMENT_CHUNKS' not in globals() or not DOCUMENT_CHUNKS:
        return {'answer': 'No hay documentos cargados para buscar.', 'citations': []}
    
    q = query.lower()
    scored = []
    for c in DOCUMENT_CHUNKS:
        text = c.get('text', '') or ''
        score = text.lower().count(q)  # heurística simple
        scored.append({'text': text, 'score': score, 'metadata': {'source': c.get('source'), 'chunk_index': c.get('chunk_index')}})
    scored = sorted(scored, key=lambda x: x['score'], reverse=True)[:k]
    
    if not scored:
        return {'answer': 'No se encontraron documentos relevantes.', 'citations': []}
    
    context = '\n'.join([s['text'][:500] for s in scored])
    prompt = f"Eres un asistente legal colombiano. Responde en español basado en el contexto proporcionado.\n\nContexto:\n{context}\n\nPregunta: {query}\n\nRespuesta:"
    
    try:
        answer = llm.invoke(prompt).content
    except Exception as e:
        answer = f"Error al invocar LLM: {e}"
    
    return {'answer': answer, 'citations': []}

# --- Agente React Básico ---
def buscar_en_leyes_normativos(query: str) -> str:
    """Busca en la base de datos legal usando RAG."""
    res = rag_answer(query, k=3)
    return f"Respuesta (RAG): {res['answer']}"

tools = [
    Tool(
        name="BusquedaLegalRAG",
        func=buscar_en_leyes_normativos,
        description="Usa esta herramienta cuando la pregunta se relacione con leyes o documentos cargados."
    ),
    Tool(
        name="LLMGeneral",
        func=lambda q: llm.invoke(q).content,
        description="Usa esta herramienta cuando la pregunta NO sea legal o no requiera buscar documentos."
    )
]

# --- Inicializamos agente con tipo ReAct ---
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,   # muestra el razonamiento paso a paso
    max_iterations=3,
)

# --- Ejemplo de uso ---
print("Ejemplo 1: Pregunta legal (usa RAG)")
respuesta1 = agent.invoke("¿Qué promedio se necesita para graduarse en la Universidad de Pamplona?")
print(respuesta1)

print("\nEjemplo 2: Pregunta general (usa LLM)")
respuesta2 = agent.invoke("¿Quién fue Gabriel García Márquez?")
print(respuesta2)

Ejemplo 1: Pregunta legal (usa RAG)


> Entering new AgentExecutor chain...
Thought: La pregunta es sobre el promedio de graduación en la Universidad de Pamplona. Esta es una pregunta de conocimiento general sobre la universidad y no requiere buscar en documentos legales o específicos cargados. Usaré la herramienta LLMGeneral.
Action: LLMGeneral
Action Input: Promedio para graduarse en la Universidad de PamplonaThought: La pregunta es sobre el promedio de graduación en la Universidad de Pamplona. Esta es una pregunta de conocimiento general sobre la universidad y no requiere buscar en documentos legales o específicos cargados. Usaré la herramienta LLMGeneral.
Action: LLMGeneral
Action Input: Promedio para graduarse en la Universidad de Pamplona
Observation: El promedio para graduarse en la Universidad de Pamplona es **determinado por cada Facultad y programa académico**. No existe una calificación única estándar para todos los estudiantes en toda la universidad.

Generalmente, lo que d

In [23]:
# ===== CELDA: Definir retrieve + rag_answer + re-crear agente ReAct =====
import textwrap

# 1) Comprobaciones básicas
if 'llm' not in globals():
    raise NameError("No existe la variable 'llm'. Inicializa tu LLM (ChatGroq/ChatOpenAI) antes de ejecutar esta celda.")

# Asegurar que vectordb existe (aunque sea None)
if 'vectordb' not in globals():
    vectordb = None
    print("⚠️ vectordb no definido, se usará búsqueda por keywords")

try:
    has_vectordb = vectordb is not None
except:
    has_vectordb = False

has_chunks = ('DOCUMENT_CHUNKS' in globals() and bool(DOCUMENT_CHUNKS))

# 2) Funciones de recuperación (vectordb o fallback por keywords)
def retrieve_by_keywords(query: str, k: int = 4):
    """Fallback simple: cuenta ocurrencias en DOCUMENT_CHUNKS."""
    if not has_chunks:
        return []
    q = query.lower()
    words = q.split()  # dividir en palabras para mejor búsqueda
    scored = []
    for c in DOCUMENT_CHUNKS:
        text = c.get('text','') or ''
        score = sum(text.lower().count(word) for word in words)  # suma de ocurrencias de cada palabra
        scored.append({'text': text, 'score': score, 'metadata': {'source': c.get('source'), 'chunk_index': c.get('chunk_index')}})
    scored = sorted(scored, key=lambda x: x['score'], reverse=True)
    return scored[:k]

def retrieve_from_vectordb(query: str, k: int = 4):
    """Intentar usar vectordb (Chroma/FAISS) si está disponible."""
    try:
        # preferimos similarity_search_with_relevance_scores si existe
        if hasattr(vectordb, "similarity_search_with_relevance_scores"):
            results = vectordb.similarity_search_with_relevance_scores(query, k=k)
            out = []
            for doc, score in results:
                text = getattr(doc, "page_content", None) or (doc.get('text') if isinstance(doc, dict) else str(doc))
                meta = getattr(doc, "metadata", None) or (doc if isinstance(doc, dict) else {})
                out.append({'text': text, 'score': float(score), 'metadata': meta})
            return out
        else:
            results = vectordb.similarity_search(query, k=k)
            out = []
            for r in results:
                text = getattr(r, "page_content", None) or (r.get('text') if isinstance(r, dict) else str(r))
                meta = getattr(r, "metadata", None) or (r if isinstance(r, dict) else {})
                out.append({'text': text, 'score': None, 'metadata': meta})
            return out
    except Exception as e:
        print("⚠️ Error buscando en vectordb:", e)
        print("Se usará fallback por keywords (DOCUMENT_CHUNKS).")
        return retrieve_by_keywords(query, k=k)

def retrieve(query: str, k: int = 4):
    return retrieve_from_vectordb(query, k) if has_vectordb else retrieve_by_keywords(query, k)

# 3) Wrapper LLM compatible con .invoke() y llamada directa
def llm_call(prompt: str, **kwargs) -> str:
    """Llama al LLM y devuelve texto; maneja distintos formatos de respuesta."""
    try:
        resp = llm.invoke(prompt, **kwargs)
        return getattr(resp, "content", None) or str(resp)
    except Exception:
        try:
            resp = llm(prompt, **kwargs)
            return getattr(resp, "content", None) or str(resp)
        except Exception as e:
            return f"[ERROR en LLM]: {e}"

# 4) rag_answer robusto: recupera, arma contexto y llama al LLM
def rag_answer(query: str, k: int = 4):
    docs = retrieve(query, k)
    if not docs:
        return {'answer': 'No se encontraron documentos relevantes.', 'citations': []}

    ctx_parts = []
    citations = []
    for d in docs:
        text = d.get('text') if isinstance(d, dict) else str(d)
        meta = d.get('metadata', {}) if isinstance(d, dict) else {}
        src = meta.get('source', meta.get('filename', 'unknown'))
        idx = meta.get('chunk_index')
        score = d.get('score')
        citations.append({'source': src, 'chunk_index': idx, 'score': score})
        snippet = textwrap.shorten(text, width=800, placeholder='...')
        ctx_parts.append(f"[{src}, chunk {idx}]\n{snippet}")

    context = "\n\n".join(ctx_parts)
    final_prompt = (
        "Eres un asistente legal colombiano. Responde basándote ÚNICAMENTE en el contexto proporcionado.\n"
        "Si el contexto no contiene información relevante, di claramente 'No encontré información específica en los documentos'.\n"
        "Responde de forma directa y concisa en español.\n\n"
        f"Contexto:\n{context}\n\n"
        f"Pregunta: {query}\n\n"
        "Respuesta:"
    )

    answer_text = llm_call(final_prompt)
    return {'answer': answer_text, 'citations': citations}

# 5) Reconstruir agente ReAct usando herramientas simplificadas
from langchain.agents import initialize_agent, Tool, AgentType

def buscar_leyes_normativas_tool(q: str) -> str:
    """Herramienta para buscar en documentos legales."""
    try:
        r = rag_answer(q, k=3)
        answer = r.get('answer', 'No se obtuvo respuesta')
        citations = r.get('citations', [])
        
        # Formato de respuesta más claro para el agente
        if citations and citations[0].get('source') != 'unknown':
            sources = ', '.join([f"{c.get('source')} (chunk {c.get('chunk_index')})" for c in citations[:2]])
            return f"{answer}\n\nFuentes consultadas: {sources}"
        return answer
    except Exception as e:
        return f"Error al buscar: {e}"

def llm_general_tool(q: str) -> str:
    """Herramienta para responder preguntas generales sin buscar en documentos."""
    try:
        prompt = f"Responde de forma breve y concisa en español:\n\n{q}"
        return llm_call(prompt)
    except Exception as e:
        return f"Error: {e}"

tools = [
    Tool(
        name="BusquedaLegalRAG",
        func=buscar_leyes_normativas_tool,
        description="Usa esta herramienta SOLO para preguntas sobre leyes, constitución, artículos legales o documentos específicos cargados. Devuelve información basada en los documentos indexados."
    ),
    Tool(
        name="RespuestaDirecta",
        func=llm_general_tool,
        description="Usa esta herramienta para conceptos generales, definiciones o preguntas que NO requieren buscar en documentos específicos. Por ejemplo: definiciones generales, explicaciones de conceptos."
    )
]

# Configurar agente con límites más razonables y mejor prompt
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=5,  # Reducido para evitar bucles largos
    early_stopping_method="generate",  # Forzar respuesta si llega al límite
    handle_parsing_errors=True  # Manejar errores de parsing
)

print("✅ Sistema RAG configurado correctamente")
print(f"   - Modo vectordb: {'Sí' if has_vectordb else 'No (usando keywords)'}")
print(f"   - Chunks disponibles: {len(DOCUMENT_CHUNKS) if has_chunks else 0}")
print("   - Agente ReAct listo con 2 herramientas")
print("\nPrueba con: agent.run('tu pregunta aquí')")

✅ Sistema RAG configurado correctamente
   - Modo vectordb: Sí
   - Chunks disponibles: 165
   - Agente ReAct listo con 2 herramientas

Prueba con: agent.run('tu pregunta aquí')


In [24]:
# ===== CELDA: Inicializar vectordb (si no existe aún) =====
# Esta celda asegura que vectordb existe como variable global
# Si tienes Chroma o FAISS configurado, puedes reemplazar None con tu vectordb

try:
    _ = vectordb
    print(f"✅ vectordb ya existe: {type(vectordb)}")
except NameError:
    vectordb = None  # Inicializar como None si no existe
    print("⚠️ vectordb no estaba definido. Se inicializó como None.")
    print("   Se usará búsqueda por keywords en DOCUMENT_CHUNKS como fallback.")
    print("   Para usar embeddings, ejecuta una celda que cree vectordb (Chroma/FAISS) antes de esta.")

✅ vectordb ya existe: <class 'langchain_community.vectorstores.chroma.Chroma'>


In [25]:
# ===== CELDA: Pruebas del agente ReAct =====

# Ejemplo 1: Pregunta sobre documentos específicos (usa RAG)
print("=" * 60)
print("EJEMPLO 1: Pregunta sobre normativa (usa RAG)")
print("=" * 60)
try:
    resultado1 = agent.run("Resume brevemente las reglas para graduarse en la Universidad de Pamplona.")
    print("\n✅ RESULTADO:")
    print(resultado1)
except Exception as e:
    print(f"\n❌ ERROR: {e}")

print("\n" + "=" * 60)
print("EJEMPLO 2: Pregunta general (no requiere documentos)")
print("=" * 60)
try:
    resultado2 = agent.run("Explica brevemente qué es el habeas corpus en términos generales.")
    print("\n✅ RESULTADO:")
    print(resultado2)
except Exception as e:
    print(f"\n❌ ERROR: {e}")

# Puedes descomentar para más pruebas:
# print("\n" + "=" * 60)
# print("EJEMPLO 3: Pregunta específica de artículos")
# print("=" * 60)
# resultado3 = agent.run("¿Qué artículo de la Constitución trata sobre la libertad de expresión?")

EJEMPLO 1: Pregunta sobre normativa (usa RAG)


> Entering new AgentExecutor chain...
Thought: La pregunta solicita un resumen de las reglas de graduación de la Universidad de Pamplona. Esta información es específica de una institución académica y es probable que esté contenida en documentos legales o reglamentos internos (estatutos, reglamentos estudiantiles, etc.). Por lo tanto, necesito usar la herramienta `BusquedaLegalRAG` que está diseñada para buscar información en documentos cargados, asumiendo que los reglamentos de la universidad están indexados.

Action: BusquedaLegalRAG
Action Input: reglas para graduarse en la Universidad de PamplonaThought: La pregunta solicita un resumen de las reglas de graduación de la Universidad de Pamplona. Esta información es específica de una institución académica y es probable que esté contenida en documentos legales o reglamentos internos (estatutos, reglamentos estudiantiles, etc.). Por lo tanto, necesito usar la herramienta `BusquedaLegalRAG` 

In [26]:
# Prueba rápida individual
print("Probando el agente con una pregunta simple...")
try:
    resultado = agent.run("¿Qué es el habeas corpus?")
    print("\n" + "="*60)
    print("RESPUESTA FINAL:")
    print("="*60)
    print(resultado)
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

Probando el agente con una pregunta simple...


> Entering new AgentExecutor chain...
Thought: The user is asking for the definition of "habeas corpus," which is a general legal concept. I should use the `RespuestaDirecta` tool.
Action: RespuestaDirecta
Action Input: Qué es el habeas corpusThought: The user is asking for the definition of "habeas corpus," which is a general legal concept. I should use the `RespuestaDirecta` tool.
Action: RespuestaDirecta
Action Input: Qué es el habeas corpus
Observation: Garantía fundamental del detenido a ser presentado inmediatamente ante un juez para determinar la legalidad de su arresto.
Thought:
Observation: Garantía fundamental del detenido a ser presentado inmediatamente ante un juez para determinar la legalidad de su arresto.
Thought:Thought: The user is asking for the definition of "habeas corpus." This is a general legal concept. I should use the `RespuestaDirecta` tool to provide the definition.
Action: RespuestaDirecta
Action Input: ¿Qué es

## 🎓 Conclusiones y Consideraciones Éticas

### ✅ Logros del Proyecto

Este proyecto implementa un **asistente legal inteligente** que combina múltiples tecnologías de IA:

1. **RAG (Retrieval-Augmented Generation)**: Respuestas basadas en documentos reales, no en alucinaciones del modelo
2. **Agentes ReAct**: Capacidad de razonamiento y selección automática de herramientas
3. **Memoria conversacional**: Contexto persistente entre múltiples preguntas
4. **Interfaz intuitiva**: UI interactiva con Jupyter widgets

### 🎯 Componentes Implementados

- ✅ **Mensajes y plantillas** (PromptTemplate)
- ✅ **Parsers estructurados** (StructuredOutputParser)
- ✅ **Loaders** (local, PDF, URLs)
- ✅ **Vector Database** (Chroma con embeddings)
- ✅ **Memoria** (ConversationBufferMemory)
- ✅ **Agentes ReAct** (razonamiento paso a paso)
- ✅ **Transformación** (splitters, sintetizadores)

### ⚠️ Limitaciones Importantes

#### 1. **Cobertura Legal**
- El sistema solo conoce los documentos que se le cargan
- No tiene acceso a jurisprudencia actualizada o fallos recientes
- La legislación colombiana cambia constantemente

#### 2. **Precisión de las Respuestas**
- Los LLMs pueden cometer errores o interpretaciones incorrectas
- La calidad depende del contexto recuperado por RAG
- Puede haber "alucinaciones" si el LLM especula más allá de los documentos

#### 3. **Actualización de Datos**
- Los documentos deben actualizarse manualmente
- No hay sincronización automática con bases de datos oficiales

### ⚖️ Consideraciones Éticas y Legales

#### **⚠️ NO ES UN SUSTITUTO DE ASESORÍA LEGAL PROFESIONAL**

Este sistema es una **herramienta de consulta y apoyo**, NO un reemplazo de:
- Abogados certificados
- Consultores legales
- Notarios o entidades oficiales

#### **Responsabilidades del Usuario:**

1. **Verificación obligatoria**: Toda información debe ser validada con fuentes oficiales
2. **Uso educativo**: Ideal para aprendizaje y consultas preliminares
3. **No para decisiones críticas**: No basar decisiones legales importantes solo en este sistema
4. **Privacidad**: No ingresar información confidencial o sensible

#### **Sesgos y Limitaciones del IA:**

- Los modelos pueden tener sesgos inherentes de sus datos de entrenamiento
- Las respuestas reflejan el contenido de los documentos cargados
- La interpretación legal requiere contexto humano y experiencia

### 🔮 Mejoras Futuras

1. **Integración con APIs oficiales** del gobierno colombiano
2. **Actualización automática** de normativas
3. **Análisis de jurisprudencia** con casos reales
4. **Multimodalidad**: procesar imágenes de documentos escaneados
5. **Búsqueda jurídica avanzada** con filtros por fecha, tipo de norma, etc.
6. **Comparación de versiones** de leyes modificadas
7. **Generación de documentos** legales básicos

### 📚 Referencias y Fuentes

- **Constitución Política de Colombia 1991**
- **Reglamentos académicos** (según documentos cargados)
- **LangChain Documentation**: https://python.langchain.com/
- **Groq API**: https://console.groq.com/
- **Chroma Vector Database**: https://www.trychroma.com/

### 👥 Contribuciones

Este proyecto fue desarrollado como parte del curso de Sistemas Inteligentes.

**Equipo:** [Agregar nombres aquí]

### 📄 Licencia y Uso

Este código es de uso educativo. Los documentos legales pertenecen a sus respectivos autores y están sujetos a sus propias licencias.

---

## 🙏 Agradecimientos

Gracias por revisar este proyecto. Esperamos que sea útil como herramienta educativa y de aprendizaje sobre aplicaciones de IA en el ámbito legal.

**Recuerda:** La tecnología es una herramienta poderosa, pero la responsabilidad y ética en su uso son fundamentales.

---

<div style="background: #ffe6e6; border-left: 4px solid #ff0000; padding: 15px; margin: 20px 0;">
<strong>⚠️ DISCLAIMER LEGAL:</strong><br>
Este sistema NO proporciona asesoría legal profesional. Para asuntos legales importantes, consulte siempre con un abogado certificado. El uso de este sistema es bajo su propia responsabilidad.
</div>

In [28]:
# ===== CELDA: Sintetizador de documentos =====
import ipywidgets as widgets
from IPython.display import display, HTML

def summarize_document(doc_text, max_length=2000):
    """Genera un resumen de un documento usando el LLM."""
    # Truncar si es muy largo
    text_to_summarize = doc_text[:max_length] if len(doc_text) > max_length else doc_text
    
    prompt = f"""Eres un asistente legal especializado. 
Genera un resumen ejecutivo del siguiente documento legal en español.
El resumen debe incluir:
1. Tema principal del documento
2. Puntos clave o artículos más importantes
3. Alcance y aplicabilidad

Documento:
{text_to_summarize}

Resumen ejecutivo:"""
    
    try:
        summary = llm_call(prompt)
        return summary
    except Exception as e:
        return f"[ERROR] No se pudo generar resumen: {e}"

def summarize_all_documents():
    """Genera resúmenes de todos los documentos cargados."""
    if not DOCUMENTS:
        return "⚠️ No hay documentos cargados"
    
    summaries = []
    for i, doc in enumerate(DOCUMENTS, 1):
        doc_name = doc.get('filename', f'Documento {i}')
        doc_text = doc.get('text', '')
        
        if len(doc_text) < 100:
            summaries.append({
                'name': doc_name,
                'summary': '[Documento muy corto, sin resumen]',
                'length': len(doc_text)
            })
            continue
        
        print(f"📝 Generando resumen {i}/{len(DOCUMENTS)}: {doc_name}...")
        summary = summarize_document(doc_text, max_length=3000)
        
        summaries.append({
            'name': doc_name,
            'summary': summary,
            'length': len(doc_text)
        })
    
    return summaries

# Widget para generar resúmenes
summarize_button = widgets.Button(
    description='📋 Generar Resúmenes',
    button_style='success',
    tooltip='Genera resúmenes de todos los documentos',
    layout=widgets.Layout(width='250px', height='40px')
)

summary_output = widgets.Output(layout=widgets.Layout(
    border='1px solid #ddd',
    padding='15px',
    max_height='600px',
    overflow='auto'
))

def on_summarize_clicked(b):
    with summary_output:
        clear_output()
        
        if not DOCUMENTS:
            display(HTML('<p style="color: orange;">⚠️ No hay documentos para resumir. Carga documentos primero.</p>'))
            return
        
        display(HTML(f'<h4>🔄 Generando resúmenes de {len(DOCUMENTS)} documentos...</h4>'))
        
        summaries = summarize_all_documents()
        
        clear_output()
        display(HTML('<h3>📚 Resúmenes de Documentos</h3>'))
        
        for i, sum_data in enumerate(summaries, 1):
            html = f'''
            <div style="background: #f9f9f9; border-left: 4px solid #4CAF50; padding: 15px; margin: 15px 0;">
                <h4 style="margin-top: 0; color: #2c3e50;">
                    📄 Documento {i}: {sum_data['name']}
                </h4>
                <p style="color: #666; font-size: 0.9em;">
                    Tamaño: {sum_data['length']:,} caracteres
                </p>
                <div style="background: white; padding: 10px; border-radius: 5px; margin-top: 10px;">
                    {sum_data['summary']}
                </div>
            </div>
            '''
            display(HTML(html))
        
        display(HTML(f'<p style="color: green; margin-top: 20px;">✅ Se generaron {len(summaries)} resúmenes exitosamente.</p>'))

summarize_button.on_click(on_summarize_clicked)

# Mostrar UI
display(widgets.VBox([
    widgets.HTML('<h3>📝 Generador de Resúmenes Automáticos</h3>'),
    widgets.HTML('<p>Esta herramienta genera resúmenes ejecutivos de todos los documentos cargados usando el LLM.</p>'),
    summarize_button,
    summary_output
]))

print(f"✅ Sintetizador listo. Documentos disponibles: {len(DOCUMENTS) if DOCUMENTS else 0}")

## 📝 Sintetizador de Documentos

Esta sección permite generar resúmenes automáticos de los documentos cargados usando el LLM.

In [29]:
# ===== CELDA OPCIONAL: Loader desde URLs =====
import ipywidgets as widgets
from IPython.display import display
import requests
from bs4 import BeautifulSoup

# Widget para ingresar URLs
url_input = widgets.Text(
    value='',
    placeholder='https://ejemplo.com/documento-legal.html',
    description='URL:',
    layout=widgets.Layout(width='80%')
)

load_url_button = widgets.Button(
    description='📥 Cargar desde URL',
    button_style='info',
    layout=widgets.Layout(width='200px')
)

url_output = widgets.Output()

def load_from_url(url):
    """Carga contenido desde una URL y lo parsea."""
    try:
        print(f"📡 Descargando desde: {url}")
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        
        # Detectar tipo de contenido
        content_type = response.headers.get('content-type', '').lower()
        
        if 'html' in content_type:
            # Parsear HTML
            soup = BeautifulSoup(response.content, 'html.parser')
            # Remover scripts y styles
            for script in soup(["script", "style"]):
                script.decompose()
            text = soup.get_text()
            # Limpiar espacios múltiples
            lines = (line.strip() for line in text.splitlines())
            text = '\n'.join(line for line in lines if line)
        else:
            # Texto plano
            text = response.text
        
        return {
            'filename': url.split('/')[-1] or 'url_document',
            'text': text,
            'url': url
        }
        
    except Exception as e:
        return {
            'filename': 'error',
            'text': f'[ERROR] No se pudo cargar URL: {e}',
            'url': url
        }

def on_load_url_clicked(b):
    url = url_input.value.strip()
    
    with url_output:
        clear_output()
        
        if not url:
            print("⚠️ Por favor ingresa una URL válida")
            return
        
        if not url.startswith(('http://', 'https://')):
            print("⚠️ La URL debe comenzar con http:// o https://")
            return
        
        # Cargar documento
        doc = load_from_url(url)
        
        if '[ERROR]' in doc['text']:
            print(doc['text'])
        else:
            # Agregar a DOCUMENTS y recrear chunks
            global DOCUMENTS, DOCUMENT_CHUNKS
            DOCUMENTS.append(doc)
            
            # Recrear todos los chunks incluyendo el nuevo documento
            DOCUMENT_CHUNKS = create_chunks(DOCUMENTS, chunk_size=1000, chunk_overlap=200)
            
            print(f"✅ Documento cargado exitosamente!")
            print(f"   - Nombre: {doc['filename']}")
            print(f"   - Tamaño: {len(doc['text'])} caracteres")
            print(f"   - Total chunks actualizados: {len(DOCUMENT_CHUNKS)}")
            print(f"\n📄 Vista previa (primeros 500 chars):")
            print(doc['text'][:500] + "...")
            print(f"\n⚠️ IMPORTANTE: Vuelve a ejecutar la celda de Vector Database para re-indexar")

load_url_button.on_click(on_load_url_clicked)

# Mostrar UI
display(widgets.VBox([
    widgets.HTML('<h4>🌐 Cargar documento desde URL</h4>'),
    url_input,
    load_url_button,
    url_output
]))

print("💡 Ejemplos de URLs válidas:")
print("   - Páginas HTML de leyes o normativas")
print("   - Archivos de texto (.txt) públicos")
print("   - Documentos en formato web")

💡 Ejemplos de URLs válidas:
   - Páginas HTML de leyes o normativas
   - Archivos de texto (.txt) públicos
   - Documentos en formato web


## 📚 Loader desde URLs (Opcional)

Esta sección permite cargar documentos legales desde URLs públicas.

In [ ]:
# ===== CELDA: Tests automatizados (smoke tests) =====

print("🧪 EJECUTANDO TESTS AUTOMATIZADOS")
print("=" * 60)

test_results = []

# Test 1: Verificar carga de documentos
print("\n📝 Test 1: Carga de documentos")
try:
    assert 'DOCUMENTS' in globals(), "Variable DOCUMENTS no existe"
    assert len(DOCUMENTS) > 0, "No se cargaron documentos"
    print(f"   ✅ PASS - {len(DOCUMENTS)} documentos cargados")
    test_results.append(('Carga de documentos', True))
except AssertionError as e:
    print(f"   ❌ FAIL - {e}")
    test_results.append(('Carga de documentos', False))

# Test 2: Verificar chunks
print("\n📦 Test 2: Creación de chunks")
try:
    assert 'DOCUMENT_CHUNKS' in globals(), "Variable DOCUMENT_CHUNKS no existe"
    assert len(DOCUMENT_CHUNKS) > 0, "No se crearon chunks"
    assert all('text' in c and 'source' in c for c in DOCUMENT_CHUNKS[:5]), "Chunks mal formados"
    print(f"   ✅ PASS - {len(DOCUMENT_CHUNKS)} chunks generados correctamente")
    test_results.append(('Creación de chunks', True))
except AssertionError as e:
    print(f"   ❌ FAIL - {e}")
    test_results.append(('Creación de chunks', False))

# Test 3: Verificar embeddings/vectordb
print("\n🔢 Test 3: Vector Database")
try:
    assert 'vectordb' in globals(), "Variable vectordb no existe"
    if vectordb is not None:
        # Test de búsqueda simple
        test_query = "constitución"
        results = vectordb.similarity_search(test_query, k=2)
        assert len(results) > 0, "Vectordb no devuelve resultados"
        print(f"   ✅ PASS - Vectordb funcional con embeddings")
        test_results.append(('Vector Database', True))
    else:
        print(f"   ⚠️ SKIP - Vectordb es None (usando keywords)")
        test_results.append(('Vector Database', 'SKIP'))
except Exception as e:
    print(f"   ❌ FAIL - {e}")
    test_results.append(('Vector Database', False))

# Test 4: Verificar función retrieve
print("\n🔍 Test 4: Función de recuperación (retrieve)")
try:
    test_query = "libertad"
    results = retrieve(test_query, k=3)
    assert len(results) > 0, "retrieve() no devuelve resultados"
    assert all('text' in r for r in results), "Resultados mal formados"
    print(f"   ✅ PASS - retrieve() devolvió {len(results)} documentos")
    test_results.append(('Función retrieve', True))
except Exception as e:
    print(f"   ❌ FAIL - {e}")
    test_results.append(('Función retrieve', False))

# Test 5: Verificar RAG
print("\n🤖 Test 5: Pipeline RAG completo")
try:
    test_question = "¿Qué es la constitución?"
    result = rag_answer(test_question, k=2)
    assert 'answer' in result, "Respuesta RAG sin campo 'answer'"
    assert 'citations' in result, "Respuesta RAG sin campo 'citations'"
    assert len(result['answer']) > 10, "Respuesta demasiado corta"
    print(f"   ✅ PASS - RAG generó respuesta válida ({len(result['answer'])} chars)")
    test_results.append(('Pipeline RAG', True))
except Exception as e:
    print(f"   ❌ FAIL - {e}")
    test_results.append(('Pipeline RAG', False))

# Test 6: Verificar LLM
print("\n💬 Test 6: Conexión con LLM")
try:
    test_prompt = "Responde con una sola palabra: OK"
    response = llm_call(test_prompt)
    assert len(response) > 0, "LLM no devolvió respuesta"
    assert not response.startswith('[ERROR'), "Error al llamar LLM"
    print(f"   ✅ PASS - LLM respondió correctamente")
    test_results.append(('Conexión LLM', True))
except Exception as e:
    print(f"   ❌ FAIL - {e}")
    test_results.append(('Conexión LLM', False))

# Test 7: Verificar agente
print("\n🎭 Test 7: Agente ReAct")
try:
    assert 'agent' in globals(), "Variable agent no existe"
    # Test simple sin ejecutar (para no gastar tokens)
    assert agent is not None, "Agente no inicializado"
    assert hasattr(agent, 'run'), "Agente sin método run()"
    print(f"   ✅ PASS - Agente ReAct inicializado correctamente")
    test_results.append(('Agente ReAct', True))
except Exception as e:
    print(f"   ❌ FAIL - {e}")
    test_results.append(('Agente ReAct', False))

# Test 8: Verificar memoria
print("\n🧠 Test 8: Sistema de memoria")
try:
    assert 'memory' in globals(), "Variable memory no existe"
    assert memory is not None, "Memoria no inicializada"
    # Verificar que puede cargar/guardar
    memory.save_context({"input": "test"}, {"output": "test"})
    history = memory.load_memory_variables({})
    assert 'history' in history, "Memoria no tiene campo history"
    print(f"   ✅ PASS - Sistema de memoria funcional")
    test_results.append(('Sistema de memoria', True))
except Exception as e:
    print(f"   ❌ FAIL - {e}")
    test_results.append(('Sistema de memoria', False))

# Resumen final
print("\n" + "=" * 60)
print("📊 RESUMEN DE TESTS")
print("=" * 60)

passed = sum(1 for _, result in test_results if result is True)
failed = sum(1 for _, result in test_results if result is False)
skipped = sum(1 for _, result in test_results if result == 'SKIP')
total = len(test_results)

for test_name, result in test_results:
    symbol = "✅" if result is True else ("⚠️" if result == 'SKIP' else "❌")
    status = "PASS" if result is True else ("SKIP" if result == 'SKIP' else "FAIL")
    print(f"{symbol} {test_name}: {status}")

print("\n" + "=" * 60)
print(f"Total: {passed}/{total} tests pasados")
if skipped > 0:
    print(f"Saltados: {skipped}")
if failed > 0:
    print(f"⚠️ Fallidos: {failed} - Revisar errores arriba")
else:
    print("🎉 ¡Todos los tests pasaron exitosamente!")
print("=" * 60)

## 🧪 Tests Automatizados (Smoke Tests)

Esta sección ejecuta pruebas rápidas para verificar que todos los componentes funcionan correctamente.

In [ ]:
# ===== CELDA: Interfaz completa con widgets =====
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import time
import os

# Historial de conversación
conversation_history = []

question_input = widgets.Textarea(
    value='',
    placeholder='Escribe tu pregunta legal aquí... Ejemplo: ¿Qué artículo protege la libertad de expresión?',
    description='Pregunta:',
    layout=widgets.Layout(width='95%', height='80px'),
    style={'description_width': '80px'}
)

submit_button = widgets.Button(
    description='🔍 Consultar',
    button_style='primary',
    tooltip='Enviar pregunta al asistente',
    layout=widgets.Layout(width='200px', height='40px')
)

clear_button = widgets.Button(
    description='🗑️ Limpiar',
    button_style='warning',
    tooltip='Limpiar conversación',
    layout=widgets.Layout(width='150px', height='40px')
)

# 2. Dropdown para selección de modo
mode_dropdown = widgets.Dropdown(
    options=[
        ('Agente Automático (ReAct)', 'agent'),
        ('Solo RAG (búsqueda directa)', 'rag'),
        ('Solo LLM (sin documentos)', 'llm')
    ],
    value='agent',
    description='Modo:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='400px')
)

# 3. Checkbox para debug
debug_checkbox = widgets.Checkbox(
    value=False,
    description='Mostrar detalles técnicos (debug)',
    indent=False,
    layout=widgets.Layout(width='300px')
)

# 4. Output areas
output_area = widgets.Output(layout=widgets.Layout(
    border='1px solid #ddd',
    padding='15px',
    margin='10px 0',
    max_height='500px',
    overflow='auto'
))

debug_area = widgets.Output(layout=widgets.Layout(
    border='1px solid #ffa500',
    padding='10px',
    margin='10px 0',
    max_height='300px',
    overflow='auto'
))

# 5. Status indicator
status_label = widgets.HTML(
    value='<span style="color: green;">✓ Sistema listo</span>',
    layout=widgets.Layout(margin='10px 0')
)

# 6. Stats widget
stats_html = widgets.HTML(
    value=f'''
    <div style="background: #f0f0f0; padding: 10px; border-radius: 5px;">
        <b>📊 Estadísticas del Sistema:</b><br>
        • Documentos cargados: {len(DOCUMENTS) if 'DOCUMENTS' in globals() else 0}<br>
        • Chunks indexados: {len(DOCUMENT_CHUNKS) if 'DOCUMENT_CHUNKS' in globals() else 0}<br>
        • Modo vectordb: {'✅ Activo' if 'vectordb' in globals() and vectordb else '⚠️ Desactivado (keywords)'}<br>
        • Proveedor LLM: {os.getenv('LLM_PROVIDER', 'groq').upper()}
    </div>
    ''',
    layout=widgets.Layout(margin='10px 0')
)

# Función para formatear respuestas con HTML
def format_response(answer, citations=None, elapsed_time=None):
    html = f'<div style="background: #e8f4f8; padding: 15px; border-radius: 5px; margin: 10px 0;">'
    html += f'<p style="margin: 0; line-height: 1.6;">{answer}</p>'
    
    if citations and len(citations) > 0:
        html += '<hr style="margin: 10px 0;">'
        html += '<p style="margin: 5px 0; font-size: 0.9em;"><b>📎 Referencias:</b></p>'
        html += '<ul style="margin: 5px 0; font-size: 0.85em;">'
        for cit in citations[:3]:  # Máximo 3 citas
            source = cit.get('source', 'unknown')
            chunk = cit.get('chunk_index', '?')
            score = cit.get('score')
            score_str = f' (score: {score:.3f})' if score else ''
            html += f'<li>{source} - Fragmento #{chunk}{score_str}</li>'
        html += '</ul>'
    
    if elapsed_time:
        html += f'<p style="margin-top: 10px; font-size: 0.8em; color: #666;">⏱️ Tiempo: {elapsed_time:.2f}s</p>'
    
    html += '</div>'
    return html

# Callback principal para procesar preguntas
def on_submit_clicked(b):
    question = question_input.value.strip()
    
    if not question:
        with output_area:
            clear_output()
            display(HTML('<p style="color: orange;">⚠️ Por favor escribe una pregunta.</p>'))
        return
    
    # Actualizar status
    status_label.value = '<span style="color: orange;">⏳ Procesando...</span>'
    
    with output_area:
        clear_output()
        display(HTML(f'<div style="background: #fff3cd; padding: 10px; border-radius: 5px;"><b>❓ Tu pregunta:</b> {question}</div>'))
    
    # Limpiar debug area
    if debug_checkbox.value:
        with debug_area:
            clear_output()
            print("🔧 MODO DEBUG ACTIVADO\n")
            print(f"Modo seleccionado: {mode_dropdown.value}")
            print(f"Pregunta: {question}\n")
    
    try:
        start_time = time.time()
        mode = mode_dropdown.value
        
        # Procesar según el modo
        if mode == 'agent':
            if debug_checkbox.value:
                with debug_area:
                    print("Ejecutando agente ReAct...\n")
            result = agent.run(question)
            answer = result
            citations = []
            
        elif mode == 'rag':
            if debug_checkbox.value:
                with debug_area:
                    print("Ejecutando RAG directo...\n")
            rag_result = rag_answer(question, k=4)
            answer = rag_result.get('answer', 'No se obtuvo respuesta')
            citations = rag_result.get('citations', [])
            
            if debug_checkbox.value:
                with debug_area:
                    print(f"Documentos recuperados: {len(citations)}")
                    for i, cit in enumerate(citations[:3], 1):
                        print(f"  {i}. {cit.get('source')} (chunk {cit.get('chunk_index')})")
            
        else:  # llm mode
            if debug_checkbox.value:
                with debug_area:
                    print("Ejecutando LLM directo (sin documentos)...\n")
            prompt = f"Eres un asistente legal colombiano. Responde de forma breve:\n\n{question}"
            answer = llm.invoke(prompt).content
            citations = []
        
        elapsed = time.time() - start_time
        
        # Guardar en historial
        conversation_history.append({
            'question': question,
            'answer': answer,
            'mode': mode,
            'timestamp': time.time()
        })
        
        # Mostrar respuesta
        with output_area:
            display(HTML(format_response(answer, citations, elapsed)))
        
        # Status
        status_label.value = f'<span style="color: green;">✓ Respuesta generada ({elapsed:.2f}s)</span>'
        
        # Limpiar input
        question_input.value = ''
        
    except Exception as e:
        with output_area:
            display(HTML(f'<p style="color: red;">❌ Error: {str(e)}</p>'))
        status_label.value = '<span style="color: red;">✗ Error en el procesamiento</span>'
        
        if debug_checkbox.value:
            with debug_area:
                import traceback
                print("TRACEBACK COMPLETO:")
                print(traceback.format_exc())

def on_clear_clicked(b):
    conversation_history.clear()
    question_input.value = ''
    with output_area:
        clear_output()
        display(HTML('<p style="color: gray;">Conversación limpiada. Escribe una nueva pregunta.</p>'))
    with debug_area:
        clear_output()
    status_label.value = '<span style="color: green;">✓ Sistema listo</span>'

# Conectar callbacks
submit_button.on_click(on_submit_clicked)
clear_button.on_click(on_clear_clicked)

# Layout final
ui_container = widgets.VBox([
    widgets.HTML('<h2 style="color: #2c3e50;">🏛️ Asistente Legal Colombiano</h2>'),
    stats_html,
    widgets.HBox([mode_dropdown, debug_checkbox]),
    question_input,
    widgets.HBox([submit_button, clear_button]),
    status_label,
    output_area,
    debug_area
], layout=widgets.Layout(padding='20px'))

# Mostrar UI
display(ui_container)

print("✅ Interfaz de usuario lista. Escribe una pregunta y presiona 'Consultar'.")

✅ Interfaz de usuario lista. Escribe una pregunta y presiona 'Consultar'.


## 🎨 Interfaz de Usuario Interactiva

Esta sección implementa una UI completa con widgets de Jupyter para interactuar con el asistente legal de forma intuitiva.